In [1]:
dataset = ''
#dataset1 = dataset
from pathlib import Path
output_root = Path("result")
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [2]:
# Parameters
dataset = "rbp"


In [3]:
import pandas as pd
from lenskit import util
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lenskit.datasets import MovieLens
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare, wilcoxon
from itertools import combinations
from lenskit import util

In [4]:
output_root = Path("result")

In [5]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [6]:
dirs

[WindowsPath('result/rbp-ALS'),
 WindowsPath('result/rbp-Bias'),
 WindowsPath('result/rbp-IALS'),
 WindowsPath('result/rbp-IIE'),
 WindowsPath('result/rbp-III'),
 WindowsPath('result/rbp-Pop'),
 WindowsPath('result/rbp-UUE')]

In [7]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)

In [8]:
# Parameters
#dataset = "CAS"
#dataset = "rbp"
#dataset = "LOG"
#output_root = Path("result")
dataset

'rbp'

In [9]:
movies = pd.read_csv(r'C:\Users\srabantiguha\lk-demo-experiment\data\ml-20m\movies.csv')
movies = movies[['movieId','genres']]
movies = movies.join(movies.pop('genres').str.get_dummies('|'))
movies.rename({'movieId': 'item'}, axis=1, inplace=True)
movies

,item,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27274,131256,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27275,131258,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27276,131260,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]
dirs

[WindowsPath('result/rbp-ALS'),
 WindowsPath('result/rbp-Bias'),
 WindowsPath('result/rbp-IALS'),
 WindowsPath('result/rbp-IIE'),
 WindowsPath('result/rbp-III'),
 WindowsPath('result/rbp-Pop'),
 WindowsPath('result/rbp-UUE')]

In [11]:
def calc_new_col(row):
    
    if row <= a :
        return 'underexposed'
    
    elif row>= b:
        return 'overexposed'
    
    else:
        return 'Neutral'

In [12]:
analy = pd.DataFrame(columns = ["dataset","shape","Dist_EEL", "quantile(0.25)","quantile(0.75)","accuracy"], index= range(len(dirs)))

for p in range(len(dirs)): 
    
    #print('Dataset:-',dirs[p])
    o = dirs[p]
    df = pd.read_csv(dirs[p], sep=',')
    #print(df.shape)

    df = df[['item','difference']]
#     print('Distribution of EEL')
    print(df['difference'].describe())

    a = df.difference.quantile(0.25) 
    b = df.difference.quantile(0.75) 
#     print("a and b")
#     print(a,b)

    df["category"] = df['difference'].apply(lambda x: calc_new_col(x))

    movies1 = movies.merge(df, on = 'item', how = 'inner')
    map1 = {'underexposed':-1, 'overexposed':1, 'Neutral':0}
    movies1['category'] = movies1['category'].map(map1)
    movies1 = movies1.drop(columns=['item', 'difference'])

    X = movies1.drop(columns=['category'])
    y = movies1['category']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
    X_train.shape, X_test.shape
    logreg = LogisticRegression(random_state=42)
  
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    ac = (accuracy_score(y_test, y_pred))
    
    analy.iloc[p,0] = o
    analy.iloc[p,1] = (df.shape)
    analy.iloc[p,2] = 'Not showed'
    #analy.iloc[p,2] = df['difference'].describe()
    analy.iloc[p,3] = a
    analy.iloc[p,4] = b
    analy.iloc[p,5] = ac
    
analy1 = analy.T
analy1.columns = analy1.iloc[0]
analy1 =  analy1.drop(analy1.index[0])

count     1190.000000
mean        10.214318
std        851.393501
min      -1607.600134
25%        -31.387500
50%         -1.550000
75%         -0.387500
max      20242.093819
Name: difference, dtype: float64
count      476.000000
mean       470.372702
std       5252.131206
min      -1668.920499
25%        -74.496875
50%        -20.149996
75%         -4.940625
max      77762.587500
Name: difference, dtype: float64
count    7346.000000
mean        1.557280
std        54.427280
min      -395.023535
25%        -7.052349
50%        -3.091211
75%        -1.068400
max      1641.180406
Name: difference, dtype: float64


count    9932.000000
mean      -15.279865
std       157.399936
min     -1668.831921
25%        -6.977589
50%        -1.002728
75%         0.002433
max      9281.396226
Name: difference, dtype: float64
count     3559.000000
mean         9.827269
std        727.376311
min      -1046.530449
25%        -57.405265
50%        -17.437500
75%         -5.425000
max      18883.492831
Name: difference, dtype: float64


count     1409.000000
mean        41.645271
std       2650.199235
min       -790.828905
25%       -167.399959
50%        -89.899999
75%        -49.584375
max      73821.037500
Name: difference, dtype: float64
count     771.000000
mean      -69.899963
std       307.721705
min     -1668.962490
25%       -40.687500
50%        -0.387500
75%        -0.387500
max      5696.184321
Name: difference, dtype: float64


In [13]:
analy1

dataset,result\rbp-ALS,result\rbp-Bias,result\rbp-IALS,result\rbp-IIE,result\rbp-III,result\rbp-Pop,result\rbp-UUE
shape,"(1190, 3)","(476, 3)","(7346, 3)","(9932, 3)","(3559, 3)","(1409, 3)","(771, 3)"
Dist_EEL,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed
quantile(0.25),-31.3875,-74.496875,-7.052349,-6.977589,-57.405265,-167.399959,-40.6875
quantile(0.75),-0.3875,-4.940625,-1.0684,0.002433,-5.425,-49.584375,-0.3875
accuracy,0.513423,0.512605,0.496462,0.511075,0.496629,0.515581,0.549223


In [14]:
# ml20m = MovieLens('data/ml-20m')
# ml20m.tag_genome